In [ ]:
# example parameters
directory = '../../output.nigoni'
#name = 'SRS104400_110.fna.gz'
name = 'nigoni_canu_2018.09.14.01.fa.gz'
render='svg'

In [ ]:
if not render: render_name = None
else: render_name = render

In [ ]:
import sys
import os.path
import collections

import plotly.graph_objects as go

import sourmash
from charcoal import utils
from charcoal.figs.sourmash_sankey import GenomeSankeyFlow

In [ ]:
from IPython.display import Markdown as md
from IPython.display import display
md(f"# Charcoal summary report for genome `{name}`")

In [ ]:
import os
from charcoal import utils

In [ ]:
summary_csv = f'{directory}/stage1_genome_summary.csv'
summary = utils.CSV_DictHelper(summary_csv, 'genome')

In [ ]:
info = summary[name]

def bp(num):
    num = int(num)
    if num > 2e6:
        return f"{num / 1e6:.1f} Mbp"
    elif num > 2e3:
        return f"{num / 1e3:.1f} kb"
    else:
        return f"{num} bp"

x = []

lineage = info['lineage'].strip()
no_lineage = False
if not lineage:
    lineage = '*none available*'
    no_lineage = True
x.append(f" Genome lineage: `{lineage}`")
display(md("\n".join(x)))

if no_lineage:
    raise SystemExit(0)  

In [ ]:
x = []

x.append("### estimated clean / contaminated bp at each taxonomic rank")
for rank in ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus']:
    bad_name = f'bad_{rank}_bp'
    good_name = f'good_{rank}_bp'

    x.append(f"* {rank}: good {bp(info[good_name])} / bad {bp(info[bad_name])}")

x.append(f"""

Of the total {bp(info['total_contigs_bp'])} in the genome file,
{bp(info['noident_contigs_bp'])} could not be identified with the database used, and
{bp(info['ignored_contigs_bp'])} was ignored by sourmash due to the scaled/ksize values used.
""")
    
    
display(md("\n".join(x)))

## How clean and dirty content changes by taxon

In [ ]:
def plot_genome_area_chart(genome, summary):
    row = summary[genome]

    x=['superkingdom', 'phylum', 'class', 'order', 'family', 'genus']

    y_ignored_bp = []
    y_noident_bp = []
    y_clean_bp = []
    y_dirty_bp = []

    ignored_bp = int(row['ignored_contigs_bp'])
    noident_bp = int(row['noident_contigs_bp'])

    for rank in x:
        # adjust good_bp for unidentified bp and ignored bp.
        tax_good_bp = int(row[f'good_{rank}_bp']) - noident_bp - ignored_bp
        tax_bad_bp = int(row[f'bad_{rank}_bp'])
        # double check calculations, 'cause why not?
        assert ignored_bp + noident_bp + tax_good_bp + tax_bad_bp == int(row['total_contigs_bp'])

        y_ignored_bp.append(ignored_bp)
        y_noident_bp.append(noident_bp)
        y_clean_bp.append(tax_good_bp)
        y_dirty_bp.append(tax_bad_bp)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x, y=y_clean_bp,
        mode='lines',
        line=dict(width=0.5, color='rgb(184, 247, 212)'),
        stackgroup='one',
        name='clean bp',
        groupnorm='percent' # sets the normalization for the sum of the stackgroup
    ))
    fig.add_trace(go.Scatter(
        x=x, y=y_noident_bp,
        mode='lines',
        line=dict(width=0.5, color='rgb(111, 231, 219)'),
        stackgroup='one',
        name='no ident'
    ))
    fig.add_trace(go.Scatter(
        x=x, y=y_ignored_bp,
        mode='lines',
        line=dict(width=0.5, color='rgb(127, 166, 238)'),
        stackgroup='one',
        name='no hashes (ignored)'
    ))
    fig.add_trace(go.Scatter(
        x=x, y=y_dirty_bp,
        mode='lines',
        line=dict(width=0.5, color='rgb(131, 90, 241)'),
        stackgroup='one',
        name='dirty bp'
    ))

    fig.update_layout(
        showlegend=True,
        xaxis_type='category',
        yaxis=dict(
            type='linear',
            range=[1, 100],
            ticksuffix='%'))
    
    title = genome
    fig.update_layout(template='plotly',
                      title=title)

    return fig

In [ ]:
fig = plot_genome_area_chart(name, summary)
fig.show(render_name)

## Taxonomic assignments across ranks

In [ ]:
def make_plotly_fig(obj, title):
    "Build a plotly figure/sankey diagram."
    # make the data to go into the sankey figure.
    labels = obj.make_labels()
    src_l, dest_l, cnt_l, color_l, label_l = obj.make_lists()

    # build figure
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = labels,
          color = "blue"
        ),
        link = dict(
          source = src_l,
          target = dest_l,
          value = cnt_l,
          color = color_l,
          label = label_l,
      ))])

    if title:
        fig.update_layout(title_text=title, font_size=10)

    return fig

In [ ]:
def make_plotly_fig(obj, title):
    "Build a plotly figure/sankey diagram."
    # make the data to go into the sankey figure.
    labels = obj.make_labels()
    src_l, dest_l, cnt_l, color_l, label_l = obj.make_lists()

    # build figure
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = labels,
          color = "blue"
        ),
        link = dict(
          source = src_l,
          target = dest_l,
          value = cnt_l,
          color = color_l,
          label = label_l,
      ))])

    if title:
        fig.update_layout(title_text=title, font_size=10)

    return fig


def make_flow_fig(title, genome_lineage, contigs_info):
    obj = GenomeSankeyFlow()
    
    counts = collections.Counter()
    for contig_name, gather_info in contigs_info.items():
        contig_taxlist = gather_info.gather_tax

        # iterate over each contig match and summarize counts.              
        # note - here we can stop at first one, or track them all.          
        # note - b/c gather counts each hash only once, these               
        #     are non-overlapping                                           
        total_hashcount = 0
        for lin, hashcount in contig_taxlist:
            counts[lin] += hashcount
            total_hashcount += hashcount
            break # display only first

        # track missing => unassigned lineage
        unident = gather_info.num_hashes - total_hashcount
        counts[obj.unassigned_lin] += unident

    # extend genome_lineage ranks
    last_genome_rank = genome_lineage[-1].rank
    taxlist = list(sourmash.lca.taxlist())[:-2]
    assert taxlist[-1] == 'genus', taxlist
    
    if genome_lineage[-1].rank != 'genus':
        genome_lineage = list(genome_lineage)
        for rank in taxlist[len(genome_lineage):]:
            genome_lineage.append(sourmash.lca.LineagePair(rank=rank, name='unassigned'))
    
    # set the color of the main lineage
    genome_lineage = tuple(genome_lineage)
    obj.colors[genome_lineage] = "lightseagreen"
    
    # for phylum level disagreements, let's go with palevioletred
    for lin in counts:
        if not utils.is_lineage_match(lin, genome_lineage, 'phylum'):
            obj.colors[lin] = 'palevioletred'
            
    # assign unassigned to good lineage, maybe?
    counts[genome_lineage] += counts[obj.unassigned_lin]
    del counts[obj.unassigned_lin]
    
    obj.make_links(genome_lineage, counts)
    fig = make_plotly_fig(obj, title)
    
    return fig

In [ ]:
contigs_filename = f'{directory}/stage1/{name}.contigs-tax.json'

summary = utils.CSV_DictHelper(summary_csv, 'genome')
row = summary[name]
genome_lineage = utils.make_lineage(row.lineage)

contigs_info = utils.load_contigs_gather_json(contigs_filename)

title = sourmash.lca.display_lineage(genome_lineage)
fig = make_flow_fig(title, genome_lineage, contigs_info)
fig.show(render_name)